# template_title validation using WOA

In [ ]:
chunk_start

In [ ]:
variable = "template_variable" 
Variable = variable.title()

In [ ]:
md(f"**Surface {Variable}** was validated using the NOAA World Ocean Atlas 2018 (WOA18) dataset. The WOA18 dataset is a gridded climatology of the global ocean. The WOA18 dataset is available at 1°. The WOA18 dataset is available for download at https://www.nodc.noaa.gov/OC5/woa18/woa18data.html. The WOA18 dataset is used as a reference dataset to validate the {variable} variable from the model.") 
md(f"We only used surface data from the WOA18 dataset to validate the {variable} variable from the {Variable} model. Model data was regridded to the WOA18 grid using bilinear interpolation. The final 20 years of the model run were used for validation.")

In [ ]:
ds_model = nc.open_data(f"../../matched/woa/woa_{variable}.nc")
mask_all(ds_model)
ds_model.subset(variable = "model")
ds_model.top()
ds_model.tmean("month")
ds_model.as_missing(0)
ds_model.run()
ds_annual = ds_model.copy()
ds_annual.tmean()
ds_annual.set_longnames({"model": Variable})

In [ ]:
ds_obs = nc.open_data(f"../../matched/woa/woa_{variable}.nc")
ds_obs.top()
ds_obs.subset(variable = "observation")
ds_obs.tmean("month")
ds_obs.as_missing(0)
ds_obs.run()
ds_obs.tmean()
ds_obs.set_longnames({"observation": Variable})

In [ ]:
ds_obs = nc.open_data(f"../../matched/woa/woa_{variable}.nc")
ds_obs.top()
ds_obs.subset(variable = "observation")
ds_obs.run()
model_units = ds_model.contents.unit.values[0]
bad_unit = True
obs_units = ds_obs.contents.unit.values[0]


if jellyfish.levenshtein_distance(model_units, obs_units) <= 4:
    bad_unit = False

if bad_unit:
    if variable == "oxygen":
        obs_units = "mmol O2/m^3"
    
    if jellyfish.levenshtein_distance(model_units, obs_units) <= 4:
        bad_unit = False

if bad_unit:
    if variable == "oxygen":
        obs_units = "mmol O2/m**3"
    
    if jellyfish.levenshtein_distance(model_units, obs_units) <= 4:
        bad_unit = False

# temperature is ok. set bad_unit to False
# 
if variable == "temperature":
    bad_unit = False 

if bad_unit:
    raise ValueError(f"Unable to match units in ERSEM and NSBC {variable}")
ds_obs.regrid(ds_model)
ds_obs.run()
ds_obs.top()
ds_obs.run()
obs_mask = ds_obs.copy()
obs_mask > -1e20
mod_mask = ds_model.copy()
mod_mask > -1e20
mod_mask * obs_mask
mod_mask.run()
ds_model * mod_mask
ds_obs * mod_mask

In [ ]:
chunk_clim

In [ ]:
chunk_bias

## Can the model reproduce seasonality of template_variable?

The ability of the model to reproduce seasonality of template_variable is assessed by comparing the modelled and observed seasonal cycle of template_variable. The seasonal cycle is calculated by averaging the monthly values of template_variable over all available model years. The seasonal cycle is calculated for each grid cell. The modelled seasonal cycle is compared to the observed seasonal cycle of template_variable. The observed seasonal cycle is calculated by averaging the observed monthly values of template_variable over all available years. The seasonal cycle is calculated for each grid cell. The modelled seasonal cycle is compared to the observed seasonal cycle using the correlation coefficient between the two. The correlation coefficient is calculated for each grid cell. The correlation coefficient ranges from -1 to 1. A value of 1 indicates a perfect agreement between the modelled and observed seasonal cycle of template_variable. A value of -1 indicates a perfect disagreement between the modelled and observed seasonal cycle of template_variable. A value of 0 indicates no agreement between the modelled and observed seasonal cycle of template_variable. 


In [ ]:
ds1 = ds_model.copy()
ds1.cdo_command("setname,model")
ds1.run()
ds2 = ds_obs.copy()
ds2.cdo_command("setname,observation")
ds2.run()
ds_cor = nc.open_data([ds1.current[0], ds2.current[0]])
ds_cor.merge(match=["month"])
ds_cor.run()
ds_ts = ds_cor.copy()
ds_cor.cor_time("model", "observation")
title = f"Seasonal temporal correlation between {variable} for model and observations"
ds_cor.run()


# output to nc

if variable != "temperature":
    out = f"../../results/temporals/{variable}_cor.nc"
    if not os.path.exists(os.path.dirname(out)):
        os.makedirs(os.path.dirname(out))
    ds_cor.to_nc(out, zip = True, overwrite = True)

In [ ]:

df_cor = ds_cor.to_dataframe().reset_index()
ds_cor.pub_plot()

In [ ]:
md(f"**Figure {i_figure}**: Seasonal temporal correlation between model and observations for " + variable)
i_figure += 1

In [ ]:
md_result, i_figure = ecoval.global_regionals(ds_model, ds_obs, variable, i_figure)

In [ ]:
md_result

In [ ]:
if variable != "temperature":
    ds_annual = ds_model.copy()
    ds_annual.rename({ds_annual.variables[0]: "model"})
    ds_annual.append(ds_obs)
    ds_annual.tmean()
    ds_annual.merge("variables")
    ds_annual.rename({ds_obs.variables[0]: "observation"})
    out_dir = "../../results/annual_mean/"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    out_file = out_dir + f"annualmean_{variable}.nc"
    ds_annual.to_nc(out_file, zip = True, overwrite = True)

In [ ]:
chunk_end